### Link of the Pycaret docs
#### https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_br.xlsx', sheet_name='Sheet1')
print(df.head(10))

        date   value    liter
0 2014-12-31     NaN    70939
1 2015-12-31     NaN   141617
2 2016-12-31     NaN   200740
3 2017-12-31  255.06   412770
4 2018-12-31  259.33   720236
5 2019-12-31  266.04  1330163
6 2020-12-31  267.16  2430153
7 2021-12-31  264.94  3296386
8 2022-12-31  291.40  4141401
9 2023-12-31  301.80  4419242


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    10 non-null     datetime64[ns]
 1   value   7 non-null      float64       
 2   liter   10 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 368.0 bytes


In [4]:
df.drop(['liter'], axis = 1,inplace=True)

In [5]:
df.set_index('date',inplace=True)

In [6]:
df=df.sort_index(axis=0)
print(df.head(10))

             value
date              
2014-12-31     NaN
2015-12-31     NaN
2016-12-31     NaN
2017-12-31  255.06
2018-12-31  259.33
2019-12-31  266.04
2020-12-31  267.16
2021-12-31  264.94
2022-12-31  291.40
2023-12-31  301.80


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. 

When integer is passed it means N continuous points in the future without any gap. 

If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [7]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

ValueError: 
Time Series modeling automation relies on running statistical tests, plots, etc.
Many of these can not be run when data has missing values. 
Your target has 3 missing values and `numeric_imputation_target` is set to `None`. 
Please enable imputation to proceed. 

In [ ]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [ ]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [ ]:
top4 = exp.compare_models(n_select = 4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:05:34
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Naive Forecaster


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
naive,Naive Forecaster,2.7152,2.3428,9.9333,9.9333,0.0345,0.0359,2.8667
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.7509,2.3972,10.5779,10.5779,0.0376,0.0382,0.1267
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,2.7525,2.3937,10.4923,10.4923,0.0372,0.0379,0.0633
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,2.7574,2.4029,10.6043,10.6043,0.0377,0.0383,0.0533
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,2.7574,2.4029,10.6043,10.6043,0.0377,0.0383,0.0567
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,2.7649,2.4054,10.5806,10.5806,0.0376,0.0382,0.0533
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,2.7925,2.4334,10.7459,10.7459,0.0382,0.0388,0.2500
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,2.7965,2.4332,10.7201,10.7201,0.0382,0.0387,0.0467
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,2.7967,2.4334,10.7208,10.7208,0.0382,0.0387,0.0567
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,2.7967,2.4334,10.7208,10.7208,0.0382,0.0387,0.4033


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":10})

## llar_cds_dt


In [ ]:
llar = exp.create_model('llar_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = llar, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.6849,0.6686,3.7600,3.7600,0.0141,0.0140
1,2020,1.9631,1.7075,7.9180,7.9180,0.0299,0.0294
2,2021,5.6243,4.8326,20.1350,20.1350,0.0691,0.0716
Mean,NaT,2.7574,2.4029,10.6043,10.6043,0.0377,0.0383
SD,NaT,2.0933,1.7696,6.9497,6.9497,0.0231,0.0243


Processing:   0%|          | 0/4 [00:00<?, ?it/s]


## ada_cds_dt


In [ ]:
ada = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## lightgbm_cds_dt

In [ ]:
light = exp.create_model('lightgbm_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = light, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.6849,0.6686,3.7600,3.7600,0.0141,0.0140
1,2020,1.9435,1.6904,7.8387,7.8387,0.0296,0.0292
2,2021,5.6243,4.8326,20.1350,20.1350,0.0691,0.0716
Mean,NaT,2.7509,2.3972,10.5779,10.5779,0.0376,0.0382
SD,NaT,2.0958,1.7719,6.9600,6.9600,0.0232,0.0244


## naive

In [ ]:

naive = exp.create_model('naive')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = naive, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:06:52
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.2040,0.1991,1.1200,1.1200,0.0042,0.0042
1,2020,0.5504,0.4787,2.2200,2.2200,0.0084,0.0083
2,2021,7.3911,6.3506,26.4600,26.4600,0.0908,0.0951
Mean,NaT,2.7152,2.3428,9.9333,9.9333,0.0345,0.0359
SD,NaT,3.3094,2.8362,11.6947,11.6947,0.0399,0.0419


In [ ]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(theta, fh = 8)


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.7960,0.7770,4.3700,4.3700,0.0164,0.0162
1,2020,1.6168,1.4062,6.5210,6.5210,0.0246,0.0243
2,2021,6.6204,5.6884,23.7010,23.7010,0.0813,0.0848
Mean,NaT,3.0111,2.6239,11.5307,11.5307,0.0408,0.0418
SD,NaT,2.5741,2.1821,8.6504,8.6504,0.0289,0.0306


,y_pred
2023,293.7995
2024,296.6517
2025,299.5038
2026,302.3560
2027,305.2081
2028,308.0603
2029,310.9124
2030,313.7645


In [ ]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(ets, fh = 8)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.7219,0.7047,3.9633,3.9633,0.0148,0.0147
1,2020,1.9116,1.6626,7.7100,7.7100,0.0291,0.0287
2,2021,6.7010,5.7577,23.9894,23.9894,0.0823,0.0859
Mean,NaT,3.1115,2.7083,11.8876,11.8876,0.0421,0.0431
SD,NaT,2.5842,2.1914,8.6929,8.6929,0.0290,0.0308


,y_pred
2023,287.2864
2024,292.9906
2025,298.6948
2026,304.3990
2027,310.1032
2028,315.8074
2029,321.5116
2030,327.2158


In [ ]:
exp.plot_model(estimator = llar, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ada, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = light, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = naive, plot = 'forecast',
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
               

## LITER

In [20]:
#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_br.xlsx', sheet_name='Sheet1')
print(df.head(10))

        date   value    liter
0 2014-12-31     NaN    70939
1 2015-12-31     NaN   141617
2 2016-12-31     NaN   200740
3 2017-12-31  255.06   412770
4 2018-12-31  259.33   720236
5 2019-12-31  266.04  1330163
6 2020-12-31  267.16  2430153
7 2021-12-31  264.94  3296386
8 2022-12-31  291.40  4141401
9 2023-12-31  301.80  4419242


In [21]:
df.drop(['value'], axis = 1,inplace=True)

In [22]:
df.set_index('date',inplace=True)

In [23]:
df=df.sort_index(axis=0)
print(df.head(10))

              liter
date               
2014-12-31    70939
2015-12-31   141617
2016-12-31   200740
2017-12-31   412770
2018-12-31   720236
2019-12-31  1330163
2020-12-31  2430153
2021-12-31  3296386
2022-12-31  4141401
2023-12-31  4419242


In [24]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=2, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,liter
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(10, 1)"
5,Transformed data shape,"(10, 1)"
6,Transformed train set shape,"(8, 1)"
7,Transformed test set shape,"(2, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [25]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

In [26]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

In [27]:
top4 = exp.compare_models(n_select = 4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:18:29
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,ARIMA


Processing:   0%|          | 0/112 [00:00<?, ?it/s]

## lightgbm_cds_dt

In [16]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.9462,1.5199,490144.1910,490144.1910,0.2017,0.2243
1,2020,0.5942,0.4351,233640.2091,233640.2091,0.0709,0.0685
2,2021,0.0463,0.0358,21337.1041,21337.1041,0.0052,0.0051
Mean,NaT,0.8622,0.6636,248373.8347,248373.8347,0.0926,0.0993
SD,NaT,0.7985,0.6270,191673.0392,191673.0392,0.0817,0.0921


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

## POLYTREND

In [17]:
polytrend = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,4.4724,3.4927,1126341.4667,1126341.4667,0.4635,0.6033
1,2020,2.8316,2.0735,1113407.7143,1113407.7143,0.3378,0.4064
2,2021,2.2690,1.7564,1045496.3571,1045496.3571,0.2524,0.2889
Mean,NaT,3.1910,2.4409,1095081.8460,1095081.8460,0.3512,0.4329
SD,NaT,0.9347,0.7549,35457.5899,35457.5899,0.0867,0.1297


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

## exp_smooth


In [18]:
exp_smooth = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.9620,1.5322,494114.0214,494114.0214,0.2033,0.2263
1,2020,0.5795,0.4244,227873.4578,227873.4578,0.0691,0.0668
2,2021,0.0563,0.0436,25953.0827,25953.0827,0.0063,0.0062
Mean,NaT,0.8659,0.6667,249313.5206,249313.5206,0.0929,0.0998
SD,NaT,0.8039,0.6314,191726.2342,191726.2342,0.0822,0.0928


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

## THETA

In [19]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:55:24
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Theta Forecaster


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,3.4422,2.6882,866912.4743,866912.4743,0.3567,0.4342
1,2020,1.3088,0.9584,514630.7053,514630.7053,0.1561,0.1693
2,2021,0.8664,0.6707,399239.1080,399239.1080,0.0964,0.1013
Mean,NaT,1.8725,1.4391,593594.0959,593594.0959,0.2031,0.2349
SD,NaT,1.1246,0.8910,198923.7830,198923.7830,0.1113,0.1436


In [ ]:
exp.predict_model(theta, fh = 8)

,y_pred
2023,4.924029e+06
2024,5.316480e+06
2025,5.708932e+06
2026,6.101383e+06
2027,6.493834e+06
2028,6.886286e+06
2029,7.278737e+06
2030,7.671188e+06


## XGBOOST

In [ ]:
xgboost = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = xgboost, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})



In [ ]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.9609,1.8623,899460.9465,899460.9465,0.3701,0.4542
1,2020,2.0020,1.8009,1346269.2401,1346269.2401,0.4084,0.5132
2,2021,0.0296,0.0274,21337.1020,21337.1020,0.0052,0.0051
Mean,NaT,1.3308,1.2302,755689.0962,755689.0962,0.2612,0.3242
SD,NaT,0.9203,0.8509,550372.0265,550372.0265,0.1817,0.2269


In [ ]:
exp.predict_model(ets, fh = 8)

,y_pred
2023,4.986399e+06
2024,5.831395e+06
2025,6.676391e+06
2026,7.521387e+06
2027,8.366383e+06
2028,9.211379e+06
2029,1.005637e+07
2030,1.090137e+07


## ALL GRAPHS

In [ ]:
exp.plot_model(estimator = light, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = xgboost, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})